In [ ]:
# %jupyter notebook

In [ ]:
# Dependencies and Setup
import pandas as pd
from IPython.display import display

# Store filepath in a variable
police_data = 'Resources/police_departments.csv'
fire_data = 'Resources/usfa-registry-national.txt'
# agency_name = 'Resources/27681-0001-Data.txt'
# agency_addrs ='Resources/38773-0001-Data.txt'

# Read each of the respective files (police, fire, agency_n, agency_addrs) and store into Pandas dataframe
police_data_df = pd.read_csv(police_data)
fire_data_df = pd.read_csv(fire_data, delimiter='\t', encoding='ISO-8859-1')


In [ ]:
# Show header for both DataFrames
display(police_data_df.head(), fire_data_df.head())


In [ ]:
# Fill NaN values with an empty field (empty string) for both DataFrames
# Perhaps move this code down after doing all the refactoring.
# police_data_df = police_data_df.fillna('')
# fire_data_df = fire_data_df.fillna('')

# Verify: Display the first few rows to verify the changes
display(police_data_df.head(), fire_data_df.head())


In [ ]:
# Define dictionaries for renaming columns, and standardize column names that have same data
police_rename_dict = {
    'name': 'agency_name',
    'sworn' : 'sworn_active_persnl',
    'type' : 'agency_type'
}

fire_rename_dict = {
    'Fire dept name' : 'agency_name',
    'HQ addr1' : 'hq_addr1',
    'HQ addr2' : 'hq_addr2',
    'HQ city' : 'hq_city',
    'HQ state' : 'hq_state',
    'HQ zip' : 'hq_zip',
    'Mail addr1' : 'addr1',
    'Mail addr2' : 'addr2',
    'Mail PO box' : 'po_box',
    'Mail city' : 'city',
    'Mail state' : 'state',
    'Mail zip' : 'zip',
    'HQ phone' : 'hq_ph',
    'HQ fax' : 'hq_fax',
    'County' : 'county',
    'Dept Type' : 'agency_type',
    'Organization Type' : 'org_type',
    'Website' : 'website',
    'Active Firefighters - Career' : 'sworn_active_persnl'
}

# Rename columns in police_data_df
police_data_df.rename(columns=police_rename_dict, inplace=True)

# Rename columns in fire_data_df
fire_data_df.rename(columns=fire_rename_dict, inplace=True)

# Verify: Display the first few rows to verify the renaming
display(police_data_df.head(), fire_data_df.head())

In [ ]:
# Drop named columns
police_data_df = police_data_df.drop(['ftciv', 'ptciv', 'ptsworn'], axis=1)
fire_data_df = fire_data_df.drop(['FDID', 'Number Of Stations', 'Active Firefighters - Volunteer', 'Active Firefighters - Paid per Call', 'Non-Firefighting - Civilian', 'Non-Firefighting - Volunteer', 'Primary agency for emergency mgmt'], axis=1)

In [ ]:
# Reset index for both DataFrames to ensure unique indices
police_data_df.reset_index(drop=True, inplace=True)
fire_data_df.reset_index(drop=True, inplace=True)

# Concatenate the DataFrames vertically
merged_df = pd.concat([police_data_df, fire_data_df], ignore_index=True)

# Replace NaN values with a blank space if needed
merged_df.fillna(" ", inplace=True)

# Display the merged DataFrame
print(merged_df.head())


In [ ]:
# List the columns in the desired order
new_column_order = [
    'agency_name', 'agency_type', 'sworn_active_persnl', 'hq_addr1', 'hq_addr2',
    'hq_city', 'hq_state', 'hq_zip', 'addr1', 'addr2', 'po_box', 'city', 'state', 
    'county', 'zip', 'hq_ph', 'hq_fax', 'org_type', 'website', 'fips'
]
# Reorder the DataFrame columns
merged_df = merged_df[new_column_order]

# Display the DataFrame to check the new order
print(merged_df.head())

In [ ]:
#Save merged dataframe to a new file
merged_df.to_csv(r'C:\Users\jchan\csi360_fire_police\Resources\merged_data.csv', index=False)
